In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import os
import math
import pandas as pd

In [ ]:
driver_path = "./chromedriver/chromedriver"
driver = webdriver.Chrome(driver_path)

url = "https://map.kakao.com/"
driver.get(url)
# 파란색 동그라미 없애기 
driver.find_element(By.XPATH,'/html/body/div[10]/div').click()

In [ ]:
# 검색 키워드 값
search_word = '서울시 고물상'

In [ ]:
# 맵페이지 다시 불러오기 
driver.get(url)

# 키워드 검색하기
driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').clear()
driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').send_keys(search_word)
driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').click()
driver.implicitly_wait(10)
time.sleep(2)

# 장소 더보기 클릭
driver.find_element(By.XPATH,'//*[@id="info.search.place.more"]').click()
driver.implicitly_wait(10)
# 1page로 다시 이동
driver.find_element(By.XPATH,'//*[@id="info.search.page.no1"]').click()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#검색된 업체 갯수 확인하기, 검색페이지수와 다음페이지 갯수 확인하기 
placecnt = int(soup.find('em', id='info.search.place.cnt').text)
pagecnt = math.ceil(placecnt / 15)
nxpa_cnt = math.floor(placecnt / 15 / 5)

print(search_word+' 검색수 : ', placecnt)
print(search_word+' 페이지수 : ', pagecnt)
print(search_word+' 다음페이지수 : ', nxpa_cnt)

#등록할 업체 배열 초기화
nm = []
addr = []
phone = []


In [ ]:
for i in range(nxpa_cnt+1):
    #하단 5개 페이지 클릭하면서 업체 리스트 긁어오기 
    for j in range(5):
        try:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            #하단 페이지 클릭하기 
            driver.find_element(By.XPATH,'//*[@id="info.search.page.no'+str(j+1)+'"]').click()
            print('현재 클릭한 페이지: ',soup.find('div', id='info.search.page').find('a', id='info.search.page.no'+str(j+1)).text)
            
            driver.implicitly_wait(10)
            time.sleep(3)
            
            #검색된 내용 파싱
            placelist = soup.find('ul',id = 'info.search.place.list')
            company_cnt = len(placelist.find_all('li'))
            
            #업체 배열에 내용 append하기
            for k in range(company_cnt):
                nm.append(placelist.find_all('div', 'head_item clickArea')[k].find('a', 'link_name').text)
                addr.append(placelist.find_all('div', 'addr')[k].find('p').text)
                phone.append(placelist.find_all('span', 'phone')[k].text)
            
            driver.implicitly_wait(10)
            time.sleep(1)
        except:
            print('페이지가 없습니다.')
            driver.implicitly_wait(10)
            time.sleep(3)
            
    try:
        # 다음 페이지 넘기기 
        driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]').click()
        print('=============')
        print('다음 페이지로')
        driver.implicitly_wait(10)
        time.sleep(3)       
        
    except:
        print('다음 페이지가 없습니다.')   
        driver.implicitly_wait(10)
        time.sleep(3)

In [ ]:
print(len(nm))
print(len(addr))
print(len(phone))


In [ ]:
#배열 합치고 pandas 저장하기
data = {
    "place_name":nm,
    "address":addr,
    "phone_number":phone,
}
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df1 = df.drop_duplicates(['place_name','address'], keep='first')

In [ ]:
df1.shape

In [ ]:
df1.to_excel('./'+search_word+'.xlsx', index=False)

In [ ]:
driver.close()